<a href="https://colab.research.google.com/github/dganguli/tlon/blob/master/test_classification_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import time
import torch

import matplotlib.pyplot as plt
from torch.autograd import Variable


print("torch cuda is available: {}".format(torch.cuda.is_available()))
print("number of gpus: {}".format(torch.cuda.device_count()))
print("1st gpu is of type: {}".format(torch.cuda.get_device_name(0)))


torch cuda is available: True
number of gpus: 1
1st gpu is of type: Tesla P4


In [2]:
import sys
! git clone https://github.com/dganguli/tlon.git
%cd /content/tlon/
! git pull
sys.path.append('/content/tlon')

Cloning into 'tlon'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 123 (delta 65), reused 78 (delta 34), pack-reused 0
Receiving objects: 100% (123/123), 15.80 MiB | 17.38 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/tlon
Already up to date.


In [0]:
from cnn import CNN

cnn_real = CNN().from_save_dict('/content/tlon/trained_models/cnn_real.pth')
cnn_synt = CNN().from_save_dict('/content/tlon/trained_models/cnn_synt.pth')

In [5]:
from data import load_mnist, load_synthetic_mnist

_, test_loader_synt = load_synthetic_mnist('/content/tlon/trained_models/generator.pth')
_, test_loader_real = load_mnist('/tmp')


Extracting /tmp/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/MNIST/raw



Extracting /tmp/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/MNIST/raw



Extracting /tmp/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/MNIST/raw



Extracting /tmp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/MNIST/raw
Processing...
Done!


In [0]:
def compute_accuracy(model, test_loader):
  model.cuda()

  correct = 0
  with torch.no_grad():
      for data, target in test_loader:
          data = data.cuda()
          target = target.cuda()
          output = model(data)
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).sum()

  accuracy = 100.* correct/ len(test_loader.dataset)

  print(accuracy)
  return accuracy

In [13]:
compute_accuracy(cnn_real, test_loader_real)
compute_accuracy(cnn_real, test_loader_synt)
compute_accuracy(cnn_synt, test_loader_real)
compute_accuracy(cnn_synt, test_loader_synt);

tensor(96.9500, device='cuda:0')
tensor(97.9300, device='cuda:0')
tensor(94.0200, device='cuda:0')
tensor(98.1400, device='cuda:0')
